### imports

In [77]:
from cmath import e
from time import time
import cv2
import numpy as np
# from skimage.feature import graycomatrix, graycoprops
from skimage.measure import shannon_entropy as Entropy
from sklearn import svm
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
import os
import pandas as pd
import numpy as np
import argparse

### program arguments

In [78]:
dir="E:/CMP/NN/project"
hingeDir=dir+"/Gold_Hinge/FeaturesOutput/hinge_features.npy"
goldDir=dir+"/Gold_Hinge/FeaturesOutput/cold_features.npy"
icdarCold = dir+"/Gold_Hinge/FeaturesOutput/icdar_cold_features.npy"
icdarHinge = dir+"/Gold_Hinge/FeaturesOutput/icdar_hinge_features.npy"
icdarLabels = dir+"/Gold_Hinge/FeaturesOutput/icdar_labels.npz"
labelsDir=dir+"/Gold_Hinge/FeaturesOutput/labels.npz"

In [79]:
X_hinge=np.load(hingeDir)
X_gold=np.load(goldDir)
Y=np.load(labelsDir)['label']
label_names=np.load(labelsDir)['label_name']

X_icdar_hinge = np.load(icdarHinge)
X_icdar_cold = np.load(icdarCold)
Y_icadar_labels = np.load(icdarLabels)['label']
label_names_icdar = np.load(icdarLabels)['label_name']

X_gold = np.concatenate((X_gold, X_icdar_cold), axis=0)
X_hinge = np.concatenate((X_hinge, X_icdar_hinge), axis=0)


# Y = np.concatenate((Y,Y_icadar_labels), axis=0) 

# label_names = np.concatenate((label_names,label_names_icdar), axis=0)

# print(X_icdar_hinge.shape)
# print(X_icdar_cold.shape)
# print(Y_icadar_labels.shape)
# print(label_names_icdar.shape)
# print(Y_icadar_labels)

In [80]:
df = pd.read_csv('train_answers.csv')
icdar_classes = df['male'].values
icdar_labels = []
for i in range(len(Y_icadar_labels)):
    icdar_labels.append(icdar_classes[int(Y_icadar_labels[i])-1])
Y = np.concatenate((Y,icdar_labels), axis=0) 


In [81]:
# idx=label_names=='icdarTrainImages'
# Y[idx]=icdar_classes[int(Y[idx])]

In [82]:
# X_train, X_test, Y_train, Y_test = train_test_split(X_hinge, Y, test_size=0.2, random_state=1)
X_train, X_test, Y_train, Y_test = train_test_split(np.concatenate((X_gold,X_hinge),axis=1), Y, test_size=0.2, random_state=1)
# X_train, X_test, Y_train, Y_test = train_test_split(np.concatenate((X_icdar_cold,X_icdar_hinge),axis=1), icdar_labels, test_size=0.2, random_state=1)   
print(X_train.shape)
print(X_test.shape)

(741, 1200)
(186, 1200)


In [83]:
def getBestParamsForSVM(X_train,Y_train,X_test,Y_test,outputFileName):
    scaler = preprocessing.MinMaxScaler().fit(X_train)
    C = [ 0.01, 0.1, 1, 10, 100,1000]
    gamma = [0.0001,0.001, 0.01, 0.1, 1,'scale']
    kernel = ['linear','rbf']
    scores = []
    scores_train = []
    
    with open(outputFileName,'w') as csvfile:
        np.savetxt(csvfile, [], delimiter=',',
                   header='C,gamma,kernel,score_train,score')
    for i in range(len(C)):
        for j in range(len(gamma)):
            for k in range(len(kernel)):
                clf = SVC(C=C[i], gamma=gamma[j],kernel=kernel[k],class_weight='balanced')
                clf.fit(scaler.transform( X_train), Y_train)
                temp_train = clf.score(scaler.transform(X_train), Y_train)
                print("Accuracy on training set: {:.2f}".format(
                    temp_train))    
                scores_train.append((temp_train,i,j))
                temp = clf.score(scaler.transform( X_test), Y_test)
                print("Accuracy on test set: {:.4f}".format(
                    temp))
                scores.append((temp,i,j))
                with open(outputFileName, 'a') as csvfile:
                    np.savetxt(csvfile, np.array([[C[i],gamma[j],kernel[k],temp_train,temp]]), delimiter=',',fmt='%s')
        
    print(max(scores))
    print(max(scores_train))

In [84]:
getBestParamsForSVM(X_train,Y_train,X_test,Y_test,'svm_results.csv')
getBestParamsForSVM(X_train[:,np.arange(0,421)],Y_train,X_test[:,np.arange(0,421)],Y_test,'svm_results_cold.csv')
getBestParamsForSVM(X_train[:,np.arange(421,1200)],Y_train,X_test[:,np.arange(421,1200)],Y_test,'svm_results_hinge.csv')

Accuracy on training set: 0.73
Accuracy on test set: 0.6613
Accuracy on training set: 0.55
Accuracy on test set: 0.5699
Accuracy on training set: 0.73
Accuracy on test set: 0.6613
Accuracy on training set: 0.55
Accuracy on test set: 0.5699
Accuracy on training set: 0.73
Accuracy on test set: 0.6613
Accuracy on training set: 0.55
Accuracy on test set: 0.5699
Accuracy on training set: 0.73
Accuracy on test set: 0.6613
Accuracy on training set: 0.55
Accuracy on test set: 0.5699
Accuracy on training set: 0.73
Accuracy on test set: 0.6613
Accuracy on training set: 0.55
Accuracy on test set: 0.5699
Accuracy on training set: 0.73
Accuracy on test set: 0.6613
Accuracy on training set: 0.55
Accuracy on test set: 0.5699
Accuracy on training set: 0.86
Accuracy on test set: 0.7043
Accuracy on training set: 0.55
Accuracy on test set: 0.5699
Accuracy on training set: 0.86
Accuracy on test set: 0.7043
Accuracy on training set: 0.55
Accuracy on test set: 0.5699
Accuracy on training set: 0.86
Accuracy 

In [ ]:
# Classifier for gold features
scaler = preprocessing.MinMaxScaler().fit(X_train[:,np.arange(0,421)])
clf = SVC(C=1, gamma=0.001,kernel='linear',class_weight='balanced',random_state=1)
X = scaler.transform(X_train[:,np.arange(0,421)])
clf.fit(scaler.transform(X_train[:,np.arange(0,421)]), Y_train)
df_gold = clf.decision_function(scaler.transform(X_test[:,np.arange(0,421)]))
predictions_gold = clf.predict(scaler.transform(X_test[:,np.arange(0,421)]))
score_test_gold = clf.score(scaler.transform( X_test[:,np.arange(0,421)]), Y_test)
print("Accuracy on test set: {:.4f}".format(
    score_test_gold))
score_train_gold = clf.score(scaler.transform( X_train[:,np.arange(0,421)]), Y_train)
print("Accuracy on training set: {:.4f}".format(
    score_train_gold))


In [ ]:
# Classifier for hinge features
scaler = preprocessing.MinMaxScaler().fit(X_train[:,np.arange(421,1200)])
clf = SVC(C=10, gamma='scale',kernel='rbf',class_weight='balanced',random_state=1)
clf.fit(scaler.transform(X_train[:,np.arange(421,1200)]), Y_train)
df_hinge = clf.decision_function(scaler.transform(X_test[:,np.arange(421,1200)]))
predictions_hinge = clf.predict(scaler.transform(X_test[:,np.arange(421,1200)]))

score_test_hinge = clf.score(scaler.transform( X_test[:,np.arange(421,1200)]), Y_test)
print("Accuracy on test set: {:.4f}".format(
    score_test_hinge))
score_train_hinge = clf.score(scaler.transform( X_train[:,np.arange(421,1200)]), Y_train)
print("Accuracy on training set: {:.4f}".format(
    score_train_hinge))


In [ ]:
# using the decision function
df = []
for i in range(len(df_gold)):
    df.append(max(df_gold[i],df_hinge[i]))

df = np.array(df)

prediction_decision_function = np.where(df>=0,1,0)
accuracy = accuracy_score(Y_test, prediction_decision_function)
print("Accuracy on test set: {:.4f}".format(
    accuracy))

In [ ]:
# classifier for both
scaler = preprocessing.MinMaxScaler().fit(X_train)
clf = SVC(C=10, gamma='scale',kernel='rbf',class_weight='balanced',random_state=1)
clf.fit(scaler.transform(X_train), Y_train)
predictions_gold_hinge = clf.predict(scaler.transform(X_test))
score_test_both = clf.score(scaler.transform( X_test), Y_test)
print("Accuracy on test set: {:.4f}".format(
    score_test_both))
score_train_both = clf.score(scaler.transform( X_train), Y_train)
print("Accuracy on training set: {:.4f}".format(
    score_train_both))

In [ ]:
# Voting classifier
final_predictions = []
for i in range(len(predictions_gold)):
    male = 0
    female = 0
    
    female = (female+1) if predictions_gold[i]==0 else female
    male = (male+1) if predictions_gold[i]==1 else male

    female = (female+1) if predictions_hinge[i]==0 else female
    male = (male+1) if predictions_hinge[i]==1 else male
    
    female = (female+1) if predictions_gold_hinge[i]==0 else female
    male = (male+1) if predictions_gold_hinge[i]==1 else male


    final_predictions.append(0 if female>male else 1)


accuracy = sum(final_predictions==Y_test)/len(Y_test)
print("Accuracy on test set: {:.4f}".format(
    accuracy))    
    


In [ ]:
def getBestParamsForRandomForest(X_train,Y_train,X_test,Y_test):
    n_estimators = [10,50,100,200]
    max_depth = [2,5,10,20,50]
    min_samples_split = [2,5,10,20,50]
    min_samples_leaf = [1,2,5,10,20]
    max_features = ['auto']
    bootstrap = [True,False]
    scores = []
    scores_train = []
    with open('randomForest0.csv', 'w') as csvfile:
        np.savetxt(csvfile,[],
                   header='n_estimators,max_depth,min_samples_split,min_samples_leaf,max_features,bootstrap,score_training,score',
                   delimiter=',', fmt='%s')
    for i in range(len(n_estimators)):
        for j in range(len(max_depth)):
            for k in range(len(min_samples_split)):
                for l in range(len(min_samples_leaf)):
                    for m in range(len(max_features)):
                        for n in range(len(bootstrap)):
                                clf = RandomForestClassifier(n_estimators=n_estimators[i],max_depth=max_depth[j],min_samples_split=min_samples_split[k],
                                                             min_samples_leaf=min_samples_leaf[l],max_features=max_features[m],bootstrap=bootstrap[n],
                                                             criterion='gini',random_state=1)
                                clf.fit(X_train, Y_train)
                                temp_train = clf.score(X_train, Y_train)
                                scores_train.append((temp_train,i,j,k,l,m,n))
                                print("Accuracy on training set: {:.2f}".format(
                                    temp_train))
                                temp = clf.score( X_test, Y_test)
                                scores.append((temp,i,j,k,l,m,n))
                                print("Accuracy on test set: {:.4f}".format(
                                    temp))
                                print("\n")
                                with open('randomForest0.csv', 'a') as csvFile:
                                    np.savetxt(csvFile, np.array([[n_estimators[i],max_depth[j],min_samples_split[k],min_samples_leaf[l],max_features[m],bootstrap[n],temp_train,temp]]), delimiter=',', fmt='%s')
    print(max(scores))
    print(max(scores_train))
# getBestParamsForRandomForest(X_train,Y_train,X_test,Y_test)

In [ ]:
def getBestParamsForDT(X_train,Y_train,X_test,Y_test):
    max_depth = [2,5,10,20,50,100]
    min_samples_split = [2,5,10,20,50]
    min_samples_leaf = [1,2,5,10,20]
    max_features = ['auto','sqrt','log2']
    scores = []
    scores_train = []
    scaler = preprocessing.StandardScaler().fit(X_train)
    with open('decisionTree.csv', 'w') as csvfile:
        np.savetxt(csvfile,[],
                   header='max_depth,min_samples_split,min_samples_leaf,max_features,score_training,score',
                   delimiter=',', fmt='%s')
    for i in range(len(max_depth)):
        for j in range(len(min_samples_split)):
            for k in range(len(min_samples_leaf)):
                for l in range(len(max_features)):
                        clf = DecisionTreeClassifier(random_state=1,max_depth=max_depth[i],min_samples_split=min_samples_split[j],
                                                     min_samples_leaf=min_samples_leaf[k],max_features=max_features[l],criterion='gini')
                        clf.fit(scaler.transform(X_train), Y_train)
                        temp_train = clf.score(scaler.transform( X_train), Y_train)
                        scores_train.append((temp_train,i,j,k,l))
                        print("Accuracy on training set: {:.2f}".format(
                            temp_train))
                        temp = clf.score(scaler.transform( X_test), Y_test)
                        scores.append((temp,i,j,k,l))
                        print("Accuracy on test set: {:.4f}".format(
                            temp))
                        print("\n")
                        with open('decisionTree.csv', 'a') as csvFile:
                            np.savetxt(csvFile, np.array([[max_depth[i],min_samples_split[j],min_samples_leaf[k],max_features[l],temp_train,temp]]), delimiter=',', fmt='%s')

getBestParamsForDT(X_train,Y_train,X_test,Y_test)

In [ ]:
cs = preprocessing.MinMaxScaler().fit(X_train)
X_train = cs.transform(X_train) 
X_train = np.nan_to_num(X_train)

X_test = cs.transform(X_test) 
X_test = np.nan_to_num(X_test)

clf = SVC(kernel='rbf', verbose=True, C=10)
clf.fit(X_train, Y_train)
# print(clf.predict(X_test))
print(clf.score(X_test,Y_test))

In [ ]:
cs = preprocessing.MinMaxScaler()
X = cs.fit_transform(X_hinge) 
X = np.nan_to_num(X)
clf = SVC(kernel='rbf', verbose=True, C=10)
scores = cross_val_score(clf, X, Y, cv=10)
print(scores)
y_pred_hinge = cross_val_predict(clf, X, Y, cv=10)
# print(y_pred_hinge)

In [ ]:
cs = preprocessing.MinMaxScaler()
X = cs.fit_transform(X_gold)
X = np.nan_to_num(X)
clf = SVC(kernel='rbf', verbose=True, C=10)
scores = cross_val_score(clf, X, Y, cv=10)
print(scores)
y_pred_cold = cross_val_predict(clf, X, Y, cv=10)
print(y_pred_cold)

In [ ]:
y_pred = np.maximum(y_pred_hinge, y_pred_cold)

In [ ]:
print(sum(y_pred_hinge == Y) / float(len(Y)))